In [1]:
from PIL import Image
from numpy.fft import rfft2, irfft2
import numpy as np
from images2gif import writeGif

In [2]:
def save_dims(ft, low, high, name):
    ft2 = np.zeros_like(ft)
    # copy the frequencies from low to high but all others stay zero.
    ft2[low:high, low:high] = ft[low:high, low:high]
    save(ft2, name)

In [3]:
def save(ft, name):
    rft = irfft2(ft)
    img = Image.fromarray(rft)
    img = img.convert('L')
    img.save(name)

In [4]:
def main():
    # Convert input into grayscale and save.
    img = Image.open("./source/stt_screenshot.png")
    img = img.convert('L')
    outpath = './output/2dft/'
    img.save(outpath + 'input_gray.png')
    # Do Fourier Transform on image.
    ft = rfft2(img)
    # Take only zeroth frequency and do Inverse FT and save.
    save_dims(ft, 0, 1, outpath + 'output_0.png')
    # Take first two frequencies in both directions.
    save_dims(ft, 0, 2, outpath + 'output_1.png')
    save_dims(ft, 0, 3, outpath + 'output_2.png')
    # Take first 50% of frequencies.
    x = min(ft.shape)
    save_dims(ft, 0, x//2, outpath + 'output_50p.png')

In [5]:
def generateGif():
    ''' Generates images to be later converted to a gif.
    This requires ImageMagick:
    convert -delay 100 -loop 0 output_*.png animation.gif
    '''
    # Requires images2gif from code.google.com/p/visvis/source/browse/vvmovie/images2gif.py 
    # from images2gif import writeGif

    img = Image.open("./source/stt_screenshot.png")
    img = img.convert('L')
    outpath = './output/2dft/'
    # Resize image before any calculation.
    # size = (640,480)
    size = img.size
    img.thumbnail(size, Image.ANTIALIAS)
    ft = rfft2(img)

    images = []
    for x in range(0, max(ft.shape)):
        ft2 = np.zeros_like(ft)
        ft2[0:x, 0:x] = ft[0:x,0:x]
        rft = irfft2(ft2)
        img_out = Image.fromarray(rft).convert('L')
        fname = outpath + 'animation/output_%05d.jpg' %(x, )
        img_out.save(fname, quality=60, optimize=True)

    writeGif('animation.gif', images, duration=0.2)

In [6]:
main()
generateGif()

/tmp/ipykernel_1548720/2668318860.py:15: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img.thumbnail(size, Image.ANTIALIAS)


ValueError: max() arg is an empty sequence